In [154]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0405_sec_c0.dta"
)
print(C.shape)
C.sample(5)

(91319, 21)


,hhcode,sec,msno,scq01,scq02,scq03,scqo4,scq05,scq06,scq07,...,scq08b,scq09a,scq09b,psu,quarter,region,psu_hies,hhno,hhcode_hies,province
77386,3.241001e+09,C,4,no,no,no,NaN,NaN,NaN,NaN,...,NaN,5 had to help at home,NaN,32410013,4,urban,30710403,11,3.071040e+09,nwfp
56455,2.161216e+09,C,6,NaN,NaN,yes,0: less than class1,yes,class 1,3 private,...,7 no facility of water / toilet,NaN,NaN,21612162,3,urban,20512309,3,2.051231e+09,sindh
56471,2.161216e+09,C,4,yes,yes,yes,cl;ass 8,yes,class 9,3 private,...,7 no facility of water / toilet,NaN,NaN,21612162,3,urban,20512309,6,2.051231e+09,sindh
63041,3.081301e+09,C,9,yes,yes,yes,class 7,no,NaN,NaN,...,NaN,10 employment,NaN,30813005,2,urban,30813201,8,3.081320e+09,nwfp
49604,2.122003e+09,C,4,no,no,no,NaN,NaN,NaN,NaN,...,NaN,5 had to help at home,NaN,21220029,2,rural,21220202,6,2.122020e+09,sindh


In [155]:
C.columns

Index(['hhcode', 'sec', 'msno', 'scq01', 'scq02', 'scq03', 'scqo4', 'scq05',
       'scq06', 'scq07', 'scq08a', 'scq08b', 'scq09a', 'scq09b', 'psu',
       'quarter', 'region', 'psu_hies', 'hhno', 'hhcode_hies', 'province'],
      dtype='object')

In [156]:
# Reduce to the questions of interest (4 questions):

C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scqo4"]
C["currently_enrrolled"] = C["scq05"] # IMPORTANT!!! (counts of enrrollment)
C["why_not"] = C["scq09a"]

C_relevant = C[
    [
        "hhcode",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrrolled",
        "why_not",
        "region",
        "province"
    ]
].copy()


In [157]:
C_relevant

,hhcode,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province
0,1.001100e+09,yes,class 10,no,2 education completed,urban,punjab
1,1.001100e+09,yes,cl;ass 8,no,2 education completed,urban,punjab
2,1.001100e+09,yes,17: ma/msc,no,2 education completed,urban,punjab
3,1.001100e+09,yes,class 10,yes,NaN,urban,punjab
4,1.001100e+09,yes,12: ba/bsc,no,10 employment,urban,punjab
...,...,...,...,...,...,...,...
91314,4.262002e+09,no,NaN,NaN,15 other,rural,balochistan
91315,4.262002e+09,no,NaN,NaN,10 employment,rural,balochistan
91316,4.262002e+09,no,NaN,NaN,13 parent did not approve,rural,balochistan
91317,4.262002e+09,no,NaN,NaN,13 parent did not approve,rural,balochistan


In [158]:
B = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0405_sec_b0.dta"
)
B.sample(5)

# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['age'] = B['sbq04']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode', 'age', 'sex', 'marital_status']].copy()

In [159]:
B.sample(5)

,hhcode,sec,msno,sbq01,sbq02,sbq03,sbq04,sbq05,weight,psu,quarter,region,psu_hies,hhno,hhcode_hies,province,sex,age,marital_status
47222,2.072002e+09,B,2,female,present,2: husband/wife,42,2:currently married,1039.639038,20720017,2,rural,20720202,3,2.072020e+09,sindh,female,42,2:currently married
99684,4.231000e+09,B,7,female,present,3:son/daughter,12,1:never married,31.941999,42310001,1,urban,40610101,5,4.061010e+09,balochistan,female,12,1:never married
23045,1.182002e+09,B,5,male,present,3:son/daughter,8,1:never married,1002.083984,11820022,3,rural,11820302,15,1.182030e+09,punjab,male,8,1:never married
82387,3.212000e+09,B,8,female,present,4:grand son / grand daughter,2,1:never married,610.083008,32120003,2,rural,32120201,12,3.212020e+09,nwfp,female,2,1:never married
89641,4.032000e+09,B,9,female,present,3:son/daughter,14,1:never married,397.356995,40320003,3,rural,40120301,3,4.012030e+09,balochistan,female,14,1:never married


In [160]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [161]:
# MERGE
years04_05 = pd.merge(B_relevant, C_relevant, on=["hhcode"], how='inner', indicator=False)

# Evaluate merge 
#years04_05._merge.value_counts()

In [162]:
# Select people in school enrollment age (I would say 4-15 years old)

years04_05 = years04_05.loc[(years04_05["age"] > 4)&(years04_05["age"] <= 15)]
years04_05['year']=2004

In [163]:
# Change dtype to string (hhcode)
years04_05["hhcode"] = years04_05["hhcode"].astype("str")

In [171]:
#years04_05['subprovince']=years04_05['hhcode'].str.extract(r'(^w{3})')
years04_05['subprovince code']= years04_05['hhcode'].apply(lambda x: x[0:4])

In [172]:
years04_05["subprovince code"] = years04_05["subprovince code"].astype("int")

In [178]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0405_v2.csv"
)
strat_name.sample(5)

,province,subprovince code,subprovince
19,Punjab,1192,Kasur
17,Punjab,1172,Narowal
116,Punjab,1031,Faisalabad
74,NWFP,3242,Swabi
90,Balochistan,4172,4172


In [179]:
# MERGE
years04_05_ = pd.merge(years04_05, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years04_05_._merge.value_counts()

both          204600
left_only          0
right_only         0
Name: _merge, dtype: int64

In [182]:
years04_05_.sample(5)

,hhcode,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province_x,year,subprovince code,province_y,subprovince,_merge
41568,1172002316.0,6,female,1:never married,yes,class 5,no,3 too expansive,rural,punjab,2004,1172,Punjab,Narowal,both
111494,2142003604.0,6,male,1:never married,yes,class 5,yes,NaN,rural,sindh,2004,2142,Sindh,Mir Pur Khas,both
188973,4102000411.0,5,female,1:never married,no,NaN,NaN,1 too old / too young,rural,balochistan,2004,4102,Balochistan,4102,both
57494,1262001104.0,7,male,1:never married,yes,class 4,no,14 child not willing,rural,punjab,2004,1262,Punjab,Pakpatten,both
97910,2082005815.0,5,male,1:never married,no,NaN,NaN,3 too expansive,rural,sindh,2004,2082,Sindh,Larkana,both


In [181]:
years04_05_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years04_05_.', index_label="id")